In [1]:
# These two lines of code make your notebook aware of changes in your utils.py file without needing to restart the session each time
%load_ext autoreload
%autoreload 2

In [39]:
from IPython.display import Audio, display
import librosa as li
import torch
torch.set_grad_enabled(False)

import numpy as np
from sklearn.neighbors import NearestNeighbors
from mix_utils import normalize
import pandas as pd
import soundfile as sf
import pygame
import os
import pickle

AUDIO_DIR = 'audio2/'
MODEL_PATH = 'models/musicnet.ts'
FEATURES_FILE_PATH = 'data/features2.csv'

# Load the model
model = torch.jit.load(MODEL_PATH).eval() 

In [34]:
# Read features file path
df = pd.read_csv(FEATURES_FILE_PATH, header=None)

# Extract the audiofile names
names = df.iloc[:, 0].to_numpy()

In [37]:
encodings = []
for audiofile in names:
    x, sr = li.load(audiofile, sr = 22050)
    # Cut audio to first 16 seconds
    x = x[:16 * sr]
    x_tensor = torch.from_numpy(x).reshape(1, 1, -1)
    x_latent = model.encode(x_tensor)
    encodings.append(x_latent)
print(len(encodings))

330


In [40]:
# Save the encodings to a pickle file
with open('encodings.pkl', 'wb') as f:
    pickle.dump(encodings, f)